In [1]:
import numpy as np
from src.Utils.load_URM import load_URM
from src.Utils.cold_melting_warm_splitting import *

In [2]:
from src.Base.Evaluation.Evaluator import EvaluatorHoldout
from src.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Utils.load_ICM import load_ICM
from src.Utils.load_URM import load_URM

URM_all = load_URM("in/data_train.csv")
ICM_all = load_ICM("in/data_ICM_title_abstract.csv")
from src.Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.80)

In [3]:
import json

with open('parameter_tuning/logs/P3alphaRecommender_logs.json', 'r') as file:
    optimum = json.load(file)

optimum = optimum['params']
optimum['topK'] = int(optimum['topK'])

In [4]:
recommender = P3alphaRecommender(URM_train)

recommender.fit(**optimum, implicit=True)

P3alphaRecommender: URM Detected 81 (1.02 %) cold users.
P3alphaRecommender: URM Detected 2400 (9.24 %) cold items.


In [8]:
results=[]

for threshold in [6, 14, 40, 50, 80, 100, 200, 1500, 2000]:
    _,melting, warm = cold_melting_warm_splitting(URM_train, threshold=threshold)
    evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], verbose=False, ignore_users=np.asarray(melting)[0])

    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    results.append(result_dict[10]["MAP"])

results

TypeError: unsupported format string passed to MAP.__format__